In [20]:
import torch
from torch import nn
from torch.utils.data import DataLoader
import numpy as np
from torch.utils.data import Dataset
import pandas as pd
from tqdm import tqdm

In [21]:
class Data(Dataset):
    def __init__(self, csv_file, transform=None):

        super(Data, self).__init__()
        file = pd.read_csv(csv_file)
        self.input_data = file[['x1', 'x2']].values
        self.labels = file['y'].values.astype(int)   
          
    def __getitem__(self, index):
        data_item = self.input_data[index]
        data_item = torch.tensor(data_item).float()

        label = self.labels[index]
        return data_item, label

    def __len__(self):
        return len(self.input_data)

In [22]:
train_data = Data('train.csv')
test_data = Data('test.csv')
batch_size = 256
train_dataloader = DataLoader(train_data, batch_size=batch_size)
test_dataloader = DataLoader(test_data, batch_size=batch_size)

In [23]:
class NeuralNetwork(nn.Module):
    def __init__(self):
        super(NeuralNetwork, self).__init__()
        self.linear_relu_stack = nn.Sequential(
           nn.Linear(2, 3),
           nn.ReLU(),
           nn.Linear(3, 3),
        )
        self.initialize_weights()

        
    def forward(self, x):
        logits = self.linear_relu_stack(x)
        return logits
    
    def initialize_weights(self):
        for m in self.modules():
            if isinstance(m, nn.Linear):
                nn.init.normal_(m.weight)
                if m.bias is not None:
                     nn.init.zeros_(m.bias)  

In [45]:
device = "cuda" if torch.cuda.is_available() else "cpu"
model = NeuralNetwork().to(device)

In [46]:
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=0.01)

In [47]:
model.train()
epoches = 10
for epoch in tqdm(range(epoches)):
    train_loss, correct = 0, 0
    for X, y in train_dataloader:
        X, y = X.to(device), y.to(device)

        pred = model(X)

        loss = loss_fn(pred, y.long())

        # Backpropagation
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        # record loss
        train_loss += loss.item()
        correct += (pred.argmax(1) == y).type(torch.float).sum().item()
    size = len(train_dataloader.dataset)
    train_loss /= len(train_dataloader)
    correct /= size
    print(f" Train accuracy: {(100*correct):>0.1f}%, Avg loss: {train_loss:>8f}")

 20%|████████▊                                   | 2/10 [00:00<00:01,  7.28it/s]

 Train accuracy: 36.9%, Avg loss: 2.982260
 Train accuracy: 51.4%, Avg loss: 1.084454


 40%|█████████████████▌                          | 4/10 [00:00<00:00,  7.98it/s]

 Train accuracy: 65.8%, Avg loss: 0.823654
 Train accuracy: 73.7%, Avg loss: 0.727175


 60%|██████████████████████████▍                 | 6/10 [00:00<00:00,  8.14it/s]

 Train accuracy: 78.0%, Avg loss: 0.671246
 Train accuracy: 80.1%, Avg loss: 0.631709


 80%|███████████████████████████████████▏        | 8/10 [00:01<00:00,  8.24it/s]

 Train accuracy: 81.1%, Avg loss: 0.600908
 Train accuracy: 81.3%, Avg loss: 0.575701


100%|███████████████████████████████████████████| 10/10 [00:01<00:00,  8.04it/s]

 Train accuracy: 81.6%, Avg loss: 0.554502
 Train accuracy: 81.8%, Avg loss: 0.536353


In [49]:
model.eval()
correct = 0
# Turn off gradient descent
with torch.no_grad():
    for X, y in tqdm(test_dataloader):
        X, y = X.to(device), y.to(device)
        pred = model(X)
        correct += (pred.argmax(1) == y).type(torch.float).sum().item()
size = len(test_dataloader.dataset)
correct = correct / size
print(f" Test accuracy: {(100*correct):>0.1f}%")

100%|████████████████████████████████████████████| 8/8 [00:00<00:00, 142.25it/s]

 Test accuracy: 55.9%
